In [ ]:
!pip install --upgrade apache-beam

In [ ]:
import logging
import apache_beam as beam

root = logging.getLogger()
root.setLevel(logging.INFO)


class SplitRow(beam.DoFn):
    def process(self, element):
        return [element.split(",")]


class ByState(beam.DoFn):
    def process(self, element):
        return [(element[4], element[0])]


def log_output(o):
    logging.info(o)


In [ ]:
class PrepareOutput(beam.DoFn):
    def process(self, element):
        (state, users) = element
        return ["{state}: {qty}".format(state=state, qty=len(users))]


p1 = beam.Pipeline()

(
    p1
    | beam.io.ReadFromText("../data/data.csv", skip_header_lines=1)
    | beam.ParDo(SplitRow())
    | beam.ParDo(ByState())
    # Using GroupByKey and summ
    | beam.GroupByKey()
    | beam.ParDo(PrepareOutput())
    ###
    | beam.Map(log_output)
)

p1.run()


In [ ]:
p2 = beam.Pipeline()

(
    p2
    | beam.io.ReadFromText("../data/data.csv", skip_header_lines=1)
    | beam.ParDo(SplitRow())
    | beam.ParDo(ByState())
    # Using Combine
    | beam.CombinePerKey(beam.combiners.CountCombineFn())
    ###
    | beam.Map(log_output)
)

p2.run()
